In [ ]:
import numpy as np
import scipy.io
from scipy import signal, interpolate
from tqdm import tqdm

# ========== Parameters ==========
fs = 1000
window_len = 100
overlap = 50
step = window_len - overlap  # 50 ms hop
N_wind = 6  # ← Updated to use 6 windows (300 ms context)

# ========== Filtering Function ==========
def filter_data(raw_eeg, fs=1000):
    lowcut = 1
    highcut = 180
    numtaps = 101
    nyquist = fs / 2
    low = lowcut / nyquist
    high = highcut / nyquist
    b = signal.firwin(numtaps, [low, high], pass_zero='bandpass')
    a = [1.0]
    filtered_data = np.zeros_like(raw_eeg)
    for ch in range(raw_eeg.shape[1]):
        filtered_data[:, ch] = signal.lfilter(b, a, raw_eeg[:, ch])
    return filtered_data

# ========== Feature Extraction ==========
def get_features(filtered_window, fs=1000):
    num_samples, num_channels = filtered_window.shape
    features = np.zeros((num_channels, 6))
    freq_bands = [(5, 15), (20, 25), (75, 115), (125, 160), (160, 175)]
    freqs = np.fft.rfftfreq(num_samples, d=1/fs)
    for ch in range(num_channels):
        x = filtered_window[:, ch]
        mean_val = np.mean(x)
        fft_vals = np.fft.rfft(x)
        mag = np.abs(fft_vals)
        band_feats = []
        for band in freq_bands:
            idx = np.logical_and(freqs >= band[0], freqs <= band[1])
            band_feats.append(np.mean(mag[idx]) if np.any(idx) else 0)
        features[ch, :] = [mean_val] + band_feats
    return features

# ========== Sliding Window Features ==========
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
    num_samples, num_channels = raw_ecog.shape
    step = window_length - window_overlap
    num_windows = (num_samples - window_overlap) // step
    all_feats = []
    for i in tqdm(range(num_windows), desc="Extracting features"):
        start = i * step
        end = start + window_length
        if end > num_samples:
            break
        window = raw_ecog[start:end, :]
        filtered = filter_data(window, fs=fs)
        feats = get_features(filtered, fs=fs)
        all_feats.append(feats.flatten())
    return np.array(all_feats)

# ========== Create R Matrix ==========
def create_R_matrix(features, N_wind):
    num_windows, num_features = features.shape
    padding = np.tile(features[0], (N_wind - 1, 1))
    padded = np.vstack([padding, features])
    R = np.zeros((num_windows, 1 + N_wind * num_features))
    for i in range(num_windows):
        window_feats = padded[i:i + N_wind].flatten()
        R[i] = np.concatenate(([1], window_feats))
    return R

# ========== Downsample Glove Data ==========
def downsample_glove(glove, window_len=100, overlap=50):
    step = window_len - overlap
    num_windows = (len(glove) - overlap) // step
    glove_down = np.zeros((num_windows, glove.shape[1]))
    for i in range(num_windows):
        glove_down[i] = np.mean(glove[i*step : i*step+window_len], axis=0)
    return glove_down

# ========== Interpolate to 1000 Hz ==========
def interpolate_prediction(pred, original_len, step=50):
    x_old = np.arange(pred.shape[0]) * step
    x_new = np.arange(original_len)
    y_interp = np.zeros((original_len, pred.shape[1]))
    for i in range(pred.shape[1]):
        spline = interpolate.CubicSpline(x_old, pred[:, i], bc_type='natural')
        y_interp[:, i] = spline(x_new)
    return y_interp

# ========== Load Data ==========
train_data = scipy.io.loadmat('raw_training_data.mat')
test_data = scipy.io.loadmat('leaderboard_data.mat')

train_ecogs = train_data['train_ecog']
train_gloves = train_data['train_dg']
leaderboard_ecogs = test_data['leaderboard_ecog']

# ========== Run Per Subject ==========
interpolated_preds = {}

for subj_idx in range(3):
    print(f"\n=== Subject {subj_idx + 1} ===")
    ecog_train = train_ecogs[subj_idx].item()
    glove_train = train_gloves[subj_idx].item()
    ecog_test = leaderboard_ecogs[subj_idx].item()

    # Training
    feats_train = get_windowed_feats(ecog_train, fs, window_len, overlap)
    R_train = create_R_matrix(feats_train, N_wind)
    glove_down = downsample_glove(glove_train, window_len, overlap)
    f = np.linalg.pinv(R_train) @ glove_down

    # Prediction
    feats_test = get_windowed_feats(ecog_test, fs, window_len, overlap)
    R_test = create_R_matrix(feats_test, N_wind)
    pred_50hz = R_test @ f

    # Interpolate to 1000 Hz
    pred_1000hz = interpolate_prediction(pred_50hz, original_len=ecog_test.shape[0], step=50)
    interpolated_preds[f'pred_1000hz_{subj_idx + 1}'] = pred_1000hz

    print(f"  → Prediction shape: {pred_1000hz.shape}")

# ========== Save Predictions in Submission Format ==========
predicted_dg = np.empty((3, 1), dtype=object)
for i in range(3):
    pred = interpolated_preds[f'pred_1000hz_{i + 1}']
    predicted_dg[i, 0] = pred

scipy.io.savemat('predicted_submission.mat', {'predicted_dg': predicted_dg})
print("\n✅ Saved to predicted_submission.mat with variable 'predicted_dg'")